In [16]:
import pandas as pd
from notdiamond import NotDiamond
from loguru import logger
from dotenv import load_dotenv
load_dotenv()

from prompts import RACIAL_BIAS_SYSTEM_PROMPT
from utils import extract_response

In [12]:
df = pd.read_csv("data/results.csv")
print(df.columns)

Index(['provider', 'set', 'text', 'label', 'response', 'predicted', 'score'], dtype='object')


In [5]:
# Define the Not Diamond routing client
client = NotDiamond()

llm_providers = [
    "openai/gpt-4o-2024-08-06",
    "openai/gpt-4o-mini-2024-07-18",
    "google/gemini-1.5-pro-latest",
    "mistral/mistral-large-2407",
    "togetherai/DeepSeek-R1",
    "perplexity/sonar"
]

In [15]:
test_data = df[(df['provider'] == 'openai/gpt-4o-2024-08-06') & (df['set'] == 'test')][['text', 'label']].reset_index(drop=True)
test_data

,text,label
0,do all members of that adhere to the same inte...,0
1,pedophile what the fuck does that even mean pa...,0
2,you gotta love antisemmetic people saying jews...,0
3,just imagine how many innocent black people ar...,0
4,if you think blm is segregating you should edu...,0
5,black people are awful period failed farming e...,1
6,i feel like driving in general is a pretty big...,1
7,keep listening to josephus judaism is devil wo...,1
8,nice work dickhead im definitely the one who i...,1
9,its almost as if many gay people are often dep...,1


## Not Diamond (base)

In [22]:
# The best LLM is determined by Not Diamond based on the messages and specified models
def generate_notdiamond_response(text:str, preference_id: str = None) -> int:
    
    if preference_id:
        result, session_id, provider = client.chat.completions.create(
            messages=[ 
                {"role": "system", "content": RACIAL_BIAS_SYSTEM_PROMPT},
                {"role": "user", "content": text}  # Adjust as desired
            ],
            model=llm_providers,
            preference_id=preference_id
        )
    else:
        result, session_id, provider = client.chat.completions.create(
            messages=[ 
                {"role": "system", "content": RACIAL_BIAS_SYSTEM_PROMPT},
                {"role": "user", "content": text}  # Adjust as desired
            ],
            model=llm_providers
        )

    return extract_response(result.content) 

In [18]:
results = []
for index, row in test_data.iterrows():
    logger.info(f"Text: {row['text']}")
    predicted = generate_notdiamond_response(row['text'])
    logger.info(f"Predicted: {predicted}, Actual: {row['label']}")
    results.append(predicted == row['label'])

2025-02-24 13:59:14.984 | INFO     | __main__:<module>:3 - Text: do all members of that adhere to the same interpretation of christianity and islam that asserts being gay is immoral 
2025-02-24 13:59:21.841 | INFO     | __main__:<module>:5 - Predicted: 0, Actual: 0
2025-02-24 13:59:21.843 | INFO     | __main__:<module>:3 - Text: pedophile what the fuck does that even mean pansexual people are not pedophiles 
2025-02-24 13:59:27.230 | INFO     | __main__:<module>:5 - Predicted: 0, Actual: 0
2025-02-24 13:59:27.232 | INFO     | __main__:<module>:3 - Text: you gotta love antisemmetic people saying jews are inferior but ruling the world secretively at the same time 
2025-02-24 13:59:33.929 | INFO     | __main__:<module>:5 - Predicted: 1, Actual: 0
2025-02-24 13:59:33.930 | INFO     | __main__:<module>:3 - Text: just imagine how many innocent black people are in jail because of racist cops like this 
2025-02-24 13:59:40.411 | INFO     | __main__:<module>:5 - Predicted: 1, Actual: 0
2025-02-

In [20]:
sum(results)

6

## NotDiamond (Trained)

In [23]:
results = []
for index, row in test_data.iterrows():
    logger.info(f"Text: {row['text']}")
    predicted = generate_notdiamond_response(row['text'], preference_id = "935e585a-2a63-411d-a4da-b103ac4e6422")
    logger.info(f"Predicted: {predicted}, Actual: {row['label']}")
    results.append(predicted == row['label'])

2025-02-24 14:03:07.478 | INFO     | __main__:<module>:3 - Text: do all members of that adhere to the same interpretation of christianity and islam that asserts being gay is immoral 
2025-02-24 14:03:13.594 | INFO     | __main__:<module>:5 - Predicted: 0, Actual: 0
2025-02-24 14:03:13.595 | INFO     | __main__:<module>:3 - Text: pedophile what the fuck does that even mean pansexual people are not pedophiles 
2025-02-24 14:03:21.389 | INFO     | __main__:<module>:5 - Predicted: 0, Actual: 0
2025-02-24 14:03:21.390 | INFO     | __main__:<module>:3 - Text: you gotta love antisemmetic people saying jews are inferior but ruling the world secretively at the same time 
2025-02-24 14:03:30.355 | INFO     | __main__:<module>:5 - Predicted: 1, Actual: 0
2025-02-24 14:03:30.355 | INFO     | __main__:<module>:3 - Text: just imagine how many innocent black people are in jail because of racist cops like this 
2025-02-24 14:03:36.565 | INFO     | __main__:<module>:5 - Predicted: 0, Actual: 0
2025-02-

In [24]:
sum(results)

7